# Django Caching Guide
Comprehensive guide to caching in Django applications, including setup, usage, invalidation, monitoring, and best practices.

## 1. Why Use Caching?
- Improve performance and reduce latency
- Reduce database and API load
- Enhance user experience
- Enable scalability for high-traffic sites

## 2. Types of Caching
- In-memory (Memcached, Redis)
- File-based
- Database
- Distributed (ElastiCache, Ignite)
- Per-view, per-template, per-object

## 3. Django Cache Setup
- Configure cache backend in `settings.py`
- Example: Redis

In [ ]:
# settings.py
CACHES = {
    'default': {
        'BACKEND': 'django_redis.cache.RedisCache',
        'LOCATION': 'redis://127.0.0.1:6379/1',
    }
}

## 4. Basic Usage
- Use Django's cache API for storing and retrieving data
- Example functions:

In [ ]:
from django.core.cache import cache

def get_cached_data(key):
    return cache.get(key)

def set_cached_data(key, value, timeout=None):
    cache.set(key, value, timeout)

def delete_cached_data(key):
    cache.delete(key)

## 5. View Caching
- Cache entire views or specific methods
- Use decorators for FBV and CBV

In [ ]:
from django.views.decorators.cache import cache_page
from django.utils.decorators import method_decorator
from django.views import View

@cache_page(60 * 15)
def my_view(request):
    # ...
    return render(request, 'template.html')

@method_decorator(cache_page(60 * 10), name='dispatch')
class MyCachedView(View):
    def get(self, request):
        return render(request, 'template.html')

## 6. Template Fragment Caching
- Cache expensive template fragments
- Use `{% load cache %}` and `{% cache %}` tags

```django
{% load cache %}
{% cache 300 sidebar request.user.pk %}
    <!-- Sidebar content -->
{% endcache %}
```

## 7. Advanced Patterns
- Custom cache keys
- Per-user, per-object caching
- Caching with signals for invalidation

In [ ]:
from django.db.models.signals import post_save, post_delete
from django.dispatch import receiver
from .models import MyModel

@receiver(post_save, sender=MyModel)
def invalidate_cache_on_save(sender, instance, **kwargs):
    cache_key = f'model_{instance.id}'
    cache.delete(cache_key)

@receiver(post_delete, sender=MyModel)
def invalidate_cache_on_delete(sender, instance, **kwargs):
    cache_key = f'model_{instance.id}'
    cache.delete(cache_key)

## 8. Monitoring and Performance
- Monitor cache hit/miss rates
- Use Django Debug Toolbar or custom logging
- Example logging config:

In [ ]:
# settings.py
LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'handlers': {
        'cache_file': {
            'level': 'INFO',
            'class': 'logging.FileHandler',
            'filename': 'logs/cache.log',
        },
    },
    'loggers': {
        'django.cache': {
            'handlers': ['cache_file'],
            'level': 'INFO',
            'propagate': False,
        },
    },
}

## 9. Testing Caching
- Test cache logic with Django's TestCase
- Use override_settings for cache backend
- Example test:

In [ ]:
from django.test import TestCase, override_settings
from django.core.cache import cache

class CacheTest(TestCase):
    @override_settings(CACHES={
        'default': {
            'BACKEND': 'django.core.cache.backends.locmem.LocMemCache',
        }
    })
    def test_cache_set_and_get(self):
        cache.set('my_key', 'my_value', 60)
        value = cache.get('my_key')
        self.assertEqual(value, 'my_value')

## 10. Limitations and Best Practices
- Stale data risk
- Cache size limits
- Complexity in invalidation
- Monitor and tune cache settings
- Use appropriate backend for your workload
- Document cache usage and invalidation logic
- Regularly review cache hit/miss rates and optimize